In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Задание 1.
Обязательная часть  
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.  
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово.  
Эти слова определяем в начале кода в переменной, например:  
KEYWORDS = ['python', 'парсинг']  

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).  
В итоге должен формироваться датафрейм вида  <дата> - <заголовок> - <ссылка>  

In [520]:
KEYWORDS = ['python', 'парсинг']

In [521]:
res = requests.get('http://habr.com/ru/all/')
res_soup = BeautifulSoup(res.text, 'html.parser')

In [522]:
#данные со страницы с постами
page = res_soup.find_all('div', class_='posts_list')

In [523]:
# получим данные по времени поста
date = list(map(lambda x: x.find_all('span', class_='post__time'), page))
#date = res_soup.findAll('span', {'class': 'post__time'}) еще вариант решения
date[0][0].get_text()

'сегодня в 10:21'

In [524]:
#получим данные по заголовку поста
lable = list(map(lambda x: x.find_all('h2', class_='post__title'), page))
lable[0][0].get_text().replace('\n', '')

'Гигантский ленивец — первое домашнее животное?'

In [525]:
#получим ссылку постов
link = list(map(lambda x: x.find('a').get('href'), lable[0]))
link[0]

'https://habr.com/ru/post/537748/'


Дополнительная часть (необязательная)  
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.  
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>  

In [526]:
res_text = requests.get('https://habr.com/ru/company/skillfactory/blog/537522/')
res_soup_text = BeautifulSoup(res_text.text, 'html.parser')

In [548]:
#получим данные их блока с текстом
text = res_soup_text.find_all('div', class_='post__text post__text-html post__text_v1')

In [549]:
#функция
def get_info_from_hubr(res):
    df = pd.DataFrame(columns=['date', 'lable', 'link', 'text'])
    date_ = []
    lable_ = []
    link_ = []
    text_ = []
    res_soup = BeautifulSoup(res.text, 'html.parser')
    #данные со страницы с постами
    page = res_soup.find_all('div', class_='posts_list')
    
    # получим данные по времени поста
    date = list(map(lambda x: x.find_all('span', class_='post__time'), page))
    for i in date[0]:
        date_.append(i.get_text())
    df['date'] = date_
    
    #получим данные по заголовку поста
    lable = list(map(lambda x: x.find_all('h2', class_='post__title'), page))
    for i in lable[0]:
        lable_.append(i.get_text().replace('\n', ''))
    df['lable'] = lable_
    
    #получим ссылку постов
    link = list(map(lambda x: x.find('a').get('href'), lable[0]))
    for i in link:
        link_.append(i)
    df['link'] = link_
    
    for i in link:
        link_.append(i)
    for i in link:
        res_text = requests.get(i)
        res_soup_text = BeautifulSoup(res_text.text, 'html.parser')
        #получим данные их блока с текстом
        text = res_soup_text.find_all('div', class_='post__text post__text-html post__text_v1')
        if text == []:
            text_.append('Nan')
        elif text != '':
            text = res_soup_text.find_all('div', class_='post__text post__text-html post__text_v1')
            text_.append(text[0].get_text())
    df['text'] = text_
    #добавил фильтрацию по заголовку
    df = df[(df['lable'].str.contains(KEYWORDS[0] or KEYWORDS[1]))]
    return df

In [550]:
get_info_from_hubr(res)

,date,lable,link,text
16,вчера в 22:37,Многопоточное скачивание файлов с ftp python-с...,https://habr.com/ru/post/537774/,Nan


### Задание 2.
Обязательная часть  
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck.   Список email-ов задаем переменной в начале кода:  
EMAIL = [xxx@x.ru, yyy@y.com]  
В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание   утечки>  
Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.  


In [33]:
url = 'https://www.avast.com/hackcheck/'

In [34]:
res = requests.post(url)

In [35]:
res_soup = BeautifulSoup(res.text, 'html.parser')

In [36]:
res_soup

<html>
<head><title>405 Not Allowed</title></head>
<body>
<center><h1>405 Not Allowed</h1></center>
<hr/><center>nginx</center>
</body>
</html>